In [2]:
from __future__ import print_function

from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

In [3]:
import sys
sys.path.insert(0, '../py')
from graviti import *

In [4]:
import numpy as np
import scipy as sp
from scipy.sparse import coo_matrix
import scipy.ndimage as ndi

from skimage.draw import polygon
from skimage import io
from skimage.measure import label, regionprops
import skimage.io
import skimage.measure
import skimage.color

import glob
import pickle
import pandas as pd
import os
import timeit
import random

import pyvips

In [5]:
import timeit
import multiprocessing
from joblib import Parallel, delayed
from datetime import datetime
from tqdm import tqdm

In [27]:
import umap
import seaborn as sns; sns.set()

In [29]:
patches = glob.glob('/home/garner1/pipelines/nucleAI/data/TCGA-A2-A0CK-01Z-00-DX1/*.pkl')

In [30]:
reducer = umap.UMAP(n_components=2,min_dist=0,n_neighbors=10)
intensity_features = []
for patch in patches[:]:
    #print(patch)
    infile = open(patch,'rb')
    lista = pickle.load(infile)
    intensity_features.extend(lista)
    infile.close()

/home/garner1/pipelines/nucleAI/data/TCGA-A2-A0CK-01Z-00-DX1/52001_36001_4000_4000_0.2456_1-features.csv.intensity_features.pkl
/home/garner1/pipelines/nucleAI/data/TCGA-A2-A0CK-01Z-00-DX1/32001_44001_4000_4000_0.2456_1-features.csv.intensity_features.pkl
/home/garner1/pipelines/nucleAI/data/TCGA-A2-A0CK-01Z-00-DX1/12001_16001_4000_4000_0.2456_1-features.csv.intensity_features.pkl
/home/garner1/pipelines/nucleAI/data/TCGA-A2-A0CK-01Z-00-DX1/44001_16001_4000_4000_0.2456_1-features.csv.intensity_features.pkl
/home/garner1/pipelines/nucleAI/data/TCGA-A2-A0CK-01Z-00-DX1/40001_24001_4000_4000_0.2456_1-features.csv.intensity_features.pkl
/home/garner1/pipelines/nucleAI/data/TCGA-A2-A0CK-01Z-00-DX1/32001_36001_4000_4000_0.2456_1-features.csv.intensity_features.pkl
/home/garner1/pipelines/nucleAI/data/TCGA-A2-A0CK-01Z-00-DX1/28001_12001_4000_4000_0.2456_1-features.csv.intensity_features.pkl
/home/garner1/pipelines/nucleAI/data/TCGA-A2-A0CK-01Z-00-DX1/12001_36001_4000_4000_0.2456_1-features.csv

/home/garner1/pipelines/nucleAI/data/TCGA-A2-A0CK-01Z-00-DX1/52001_32001_4000_4000_0.2456_1-features.csv.intensity_features.pkl
/home/garner1/pipelines/nucleAI/data/TCGA-A2-A0CK-01Z-00-DX1/24001_8001_4000_4000_0.2456_1-features.csv.intensity_features.pkl
/home/garner1/pipelines/nucleAI/data/TCGA-A2-A0CK-01Z-00-DX1/52001_44001_4000_4000_0.2456_1-features.csv.intensity_features.pkl
/home/garner1/pipelines/nucleAI/data/TCGA-A2-A0CK-01Z-00-DX1/36001_24001_4000_4000_0.2456_1-features.csv.intensity_features.pkl
/home/garner1/pipelines/nucleAI/data/TCGA-A2-A0CK-01Z-00-DX1/60001_40001_4000_4000_0.2456_1-features.csv.intensity_features.pkl
/home/garner1/pipelines/nucleAI/data/TCGA-A2-A0CK-01Z-00-DX1/24001_24001_4000_4000_0.2456_1-features.csv.intensity_features.pkl
/home/garner1/pipelines/nucleAI/data/TCGA-A2-A0CK-01Z-00-DX1/8001_36001_4000_4000_0.2456_1-features.csv.intensity_features.pkl
/home/garner1/pipelines/nucleAI/data/TCGA-A2-A0CK-01Z-00-DX1/12001_28001_4000_4000_0.2456_1-features.csv.i

/home/garner1/pipelines/nucleAI/data/TCGA-A2-A0CK-01Z-00-DX1/48001_32001_4000_4000_0.2456_1-features.csv.intensity_features.pkl


In [ ]:
data = np.array([np.real(sp.linalg.logm(np.cov(f,rowvar=False))).flatten() for f in intensity_features if f is not None])  

logm result may be inaccurate, approximate err = 1.3285379257558228e-09
logm result may be inaccurate, approximate err = 7.092682542049197e-13


In [ ]:
embedding = reducer.fit_transform(data)
x = embedding[:,0]
y = embedding[:,1]
df_plot = pd.DataFrame(dict(x=x, y=y))

fig, ax = plt.subplots(figsize=(10,10))
ax = sns.scatterplot(x="x", y="y", data=df_plot)

In [ ]:
feature_data = np.zeros((arr_3d.shape[0]*arr_3d.shape[1],9))
idx = 0
for r in range(arr_3d.shape[0]):
    for c in range(arr_3d.shape[1]):
        feature_data[idx,:] = np.hstack((r,c,
                                    arr_3d[r,c,0],arr_3d[r,c,1],arr_3d[r,c,2],
                                    delta_x[r,c],delta_y[r,c],
                                    delta_xx[r,c],delta_yy[r,c]))
        idx += 1

print(np.corrcoef(feature_data,rowvar=False)) # get the normalized covariace matrix
print(np.cov(feature_data,rowvar=False)) # get the covariace matrix

In [ ]:
# create stain to color map
stain_color_map = htk.preprocessing.color_deconvolution.stain_color_map
print('stain_color_map:', stain_color_map, sep='\n')

# specify stains of input image
stains = ['hematoxylin',  # nuclei stain
          'eosin',        # cytoplasm stain
          'null']         # set to null if input contains only two stains

# create stain matrix
W = np.array([stain_color_map[st] for st in stains]).T

# perform standard color deconvolution
imDeconvolved = htk.preprocessing.color_deconvolution.color_deconvolution(imInput, W)

# Display results
for i in [0]:#, 1:
    plt.figure()
    plt.imshow(imDeconvolved.Stains[:, :, i])
    _ = plt.title(stains[i], fontsize=titlesize)

In [ ]:
plt.figure()
masked = np.multiply(imDeconvolved.Stains[:, :, 0],~mask)
plt.imshow(masked>1)